## Henry Code - You already have ## 

In [ ]:
## import pandas
import pandas as pd

## read file into dat aframe
df = pd.read_table('/Users/jakevazquez18/Desktop/Axel_Scripts/new_out_ccf/out.ccf', names=['barcode', 'locus'])
df = df.groupby('barcode')['locus'].apply(list).reset_index()
df['number of duplicates'] = df['locus'].str.len()
df['locus'] = df['locus'].apply(lambda x: list(set(x)))
df['number of ccfs'] = df['locus'].str.len()
df.sort_values(by='number of duplicates', ascending=False)

## view the dataframe
df

## DO NOT EDIT THIS CELL - JUST RUN TO SAVE THE VARIABLES FOR PLOTTING

In [ ]:
import numpy as np

## 50
hist, bin_edges = np.histogram(df['number of duplicates'], bins = 50, range = [1, 50])

bin_edges = np.round(bin_edges,0)

bins = list(range(1, 51))

products = []
for h, b in zip(hist, bins):
    products.append(h * b)
    
df_b4_dup = pd.DataFrame(list(zip(bins, products)),
               columns =['bins', 'read_count'])

## THIS IS WHERE YOU CAN CHANGE BIN AND RANGE VALUES

In [ ]:
## use numpy (np) to create histogram values and bin edges automatically 
hist2, bin_edges2 = np.histogram(df['number of duplicates'], bins =  3000, range = [1, 250000])

## round the bin edges up
bin_edges2 = np.round(bin_edges2,0)

## set bin edges manually if needed
#bins2 = list(range(1, 3001))

## create empty list that we will add to
products2 = []

## for each element in both histogram and bin edges list, multiply them together and add to products list
for h2, b2 in zip(hist2, bin_edges2):
    products2.append(h2 * b2)
    
## create dataframe of product
df_b4_dup2 = pd.DataFrame(list(zip(bin_edges2, products2)),
               columns =['bins', 'read_count'])

## using dictionary to convert specific columns
convert_dict_dup = {'bins': float,
                'read_count': int}

## change the type of each column by passing them through the dictionary above
df_b4_dup = df_b4_dup.astype(convert_dict_dup)

print("Hist:\n", hist2)
print("Edges:\n", bin_edges2)
print("Hist values * Bins = ", products2)
df_b4_dup2

## MATH TO CALCULATE CUMALITIVE SUM AND PERCENTAGES FOR PLOTTING

In [ ]:
## percentage curve

## 1. Calculate total # of reads (unique barcodes)
## 2. Create rolling sum column with 1st row being first row of read_count column (NOT NaN)
## 3. Create Percentage column each row of sum column divided by total number of reads * 100
## 4. Plot as line to see curve

# counting unique values by counting unique barcodes
#total_rc = len(pd.unique(df['barcode']))

## round the read count to the nearest number
df_b4_dup2['read_count'] = round(df_b4_dup2['read_count'])

## calculate the cumalitive sum of the read count column using cumsum()
df_b4_dup2['sum'] = df_b4_dup2['read_count'].cumsum()

## number of reads = total of sum column
total = sum(df_b4_dup2['sum'])

## calculate percentage by dividing each cumalitive sum by the total number of reads
df_b4_dup2['Percentage'] = (df_b4_dup2['sum'] / total) * 100


print("Number of reads :", total)
print("Total Percentage: ", sum(df_b4_dup2['Percentage']))
df_b4_dup2

## PLOT

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


## initalize subplots
fig = make_subplots(rows=1, cols=2)

## create bar chart for 50 bins
fig.add_trace(go.Bar(x=df_b4_dup['bins'], y=df_b4_dup['read_count']), 1, 1)

## create line curve second subplot
fig.add_trace(go.Scatter(x=bins2, y=df_b4_dup2['Percentage']), 1, 2)

## update y axes range if needed
#fig.update_yaxes(range=[0, 100], row = 1, col = 2)

## color
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)

## show plot
fig.show()